# ICNet: Invariance Constrained Discovery for Partial Differential Equations

## Environment Setup

This notebook requires **MindSpore version >= 2.0.0** to support new APIs including: *mindspore.jit, mindspore.jit_class, mindspore.data_sink*. Please check [MindSpore Installation](https://www.mindspore.cn/install/en) for details.

In addition, **MindFlow version >=0.1.0** is also required. If it has not been installed in your environment, please select the right version and hardware, then install it as follows.

In [4]:
mindflow_version = "0.1.0"  # update if needed
# GPU Comment out the following code if you are using NPU.
!pip uninstall -y mindflow-gpu
!pip install mindflow-gpu==$mindflow_version

# NPU Uncomment if needed.
# !pip uninstall -y mindflow-ascend
# !pip install mindflow-ascend==$mindflow_version

## Background

The physical laws described by partial differential equations are widely present in the natural environment. The calculation and simulation of physical systems rely on accurate basic equations and models. The traditional method of deriving control equations is mainly based on first principles, such as the Navier-Stokes equations based on momentum conservation. The difficulty of traditional methods lies in the fact that models and equations of complex dynamics are often difficult to derive, such as multiphase flow, neuroscience, and biological science. In the era of big data, mining control equations from data through artificial intelligence methods has become a new research idea. The existing data-driven method of discovering equations still has certain limitations. At present, there is a lack of guiding principles when constructing candidates for a complete library, and it is impossible to ensure that the discovered equations meet basic physical requirements. At the same time, when dealing with complex multidimensional systems, the candidate library is too large, and it is difficult to discover simple and accurate equations. Considering that basic physical requirements (invariance, conservation, etc.) are the cornerstones of many physical problems, it is necessary to study how to impose physical constraints in discovering equations.

## Model framework

The model framework is as shown in the following figure:

![ICNet](images/ICNet.png)

In the figure:
A. Schematic diagram of the derivation process of embedding invariance constraints into the framework of partial differential equation discovery;
B. The neural network module of partial differential equation discovery with invariance constraints uses neural network automatic differentiation to obtain the partial derivatives required to construct the invariance candidate function. The loss function includes data loss, invariance loss, and regularization loss for enhancing sparsity.

## Preparation

Before practice, ensure that MindSpore of suitable version has been correctly installed. If not, you can run the following command:

* [MindSpore installation page](https://www.mindspore.cn/install) Install MindSpore.

## Datasets Preparation

Dataset download link: [ICNet/dataset](https://download-mindspore.osinfra.cn/mindscience/mindflow/dataset/applications/research/ICNet/)。Save the dataset under path `./dataset`.

## Model Training

Import code packs.

In [ ]:
import argparse
import time
import numpy as np

import mindspore as ms
from mindspore import set_seed, context, nn
from src.network import InvarianceConstrainedNN, InvarianceConstrainedNN_STRdige
from src.datasets import read_training_data, print_pde

Setting of model-related parameters and definition of training model

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--model_name', type=str, default='ICNet')
parser.add_argument('--case', type=str, default='Kuramoto-Sivashinsky equation')
parser.add_argument('--device', type=str, default='GPU')    #default='GPU' or 'Ascend'
parser.add_argument('--device_id', type=str, default=3)
parser.add_argument('--init_steps', type=str, default=0)
parser.add_argument('--stop_steps', type=str, default=150)
parser.add_argument('--time_steps', type=str, default=50)
parser.add_argument('--load_params', type=str, default='True')
parser.add_argument('--second_path', type=str, default='pretrain')
parser.add_argument('--data_name', type=str, default='KS.mat')
parser.add_argument('--description_ks', type=str, default=['uu_x', '1', 'u_x', 'u_xx', 'u_xxx', 'u_xxxx'])
parser.add_argument('--network_size', type=int, default=[2] + 8*[40] + [1])
parser.add_argument('--learning_rate', type=int, default=[0.001, 0.0005, 1.0e-04, 1.0e-05])
parser.add_argument('--epochs', type=int, default=[30e4, 30e4, 1e4, 1e4])
parser.add_argument('--BatchNo', type=int, default=1)
parser.add_argument('--lam', type=float, default=1e-5)
parser.add_argument('--d_tol', type=float, default=1.0)
args = parser.parse_known_args()[0]

model_name = args.model_name
case = args.case
device = args.device
device_id = args.device_id
network_size = args.network_size
learning_rate = args.learning_rate
epochs = args.epochs
BatchNo = args.BatchNo
load_params = args.load_params
second_path = args.second_path
description_ks = args.description_ks
lam = args.lam
d_tol = args.d_tol

use_ascend = context.get_context(attr_key='device_target') == "Ascend"

if use_ascend:
    msfloat_type = ms.float16
else:
    msfloat_type = ms.float32

context.set_context(mode=context.GRAPH_MODE, save_graphs=False, device_target=device, device_id=device_id)

X_u_train, u_train, X_f_train = read_training_data(args)

model_pretrain = InvarianceConstrainedNN(X_u_train, u_train, X_f_train, network_size, BatchNo, use_ascend, msfloat_type)

Set the seed.

In [3]:
np.random.seed(123456)
set_seed(123456)

Code training and output results.

In [ ]:
def train(model, niter, lr):
    # Get the gradients function
    params = model.dnn.trainable_params()
    params.append(model.lambda_u)
    params.append(model.lambda_uux)

    optimizer_Adam = nn.Adam(params, learning_rate=lr)

    grad_fn = ms.value_and_grad(model.loss_fn, None, optimizer_Adam.parameters, has_aux=True)

    model.dnn.set_train()

    start_time = time.time()

    for epoch in range(1, 1+niter):
        (loss, loss_u, loss_f_u, loss_lambda_u), grads = grad_fn(model.x, model.t, model.x_f, model.t_f, model.u)

        optimizer_Adam(grads)

        if epoch % 10 == 0:
            elapsed = time.time() - start_time
            print('It: %d, Loss: %.3e, loss_u:  %.3e, loss_f:  %.3e, loss_lambda:  %.3e, Lambda_uux: %.3f, Lambda_uxx: %.3f, Lambda_uxxxx: %.3f, Time: %.2f'  %\
                    (epoch, loss.item(), loss_u.item(), loss_f_u.item(), loss_lambda_u.item(),
                     model.lambda_uux.item(), model.lambda_u[2].item(), model.lambda_u[4].item(), elapsed))

            initial_size = 5

            loss_history_Adam_Pretrain = np.empty([0])
            loss_u_history_Adam_Pretrain = np.empty([0])
            loss_f_u_history_Adam_Pretrain = np.empty([0])
            loss_lambda_u_history_Adam_Pretrain = np.empty([0])

            lambda_u_history_Adam_Pretrain = np.zeros((initial_size, 1))
            lambda_uux_history_Adam_Pretrain = np.zeros((1, 1))

            loss_history_Adam_Pretrain = np.append(loss_history_Adam_Pretrain, loss.numpy())
            lambda_u_history_Adam_Pretrain = np.append(lambda_u_history_Adam_Pretrain, model.lambda_u.numpy(), axis=1)
            loss_u_history_Adam_Pretrain = np.append(loss_u_history_Adam_Pretrain, loss_u.numpy())
            loss_f_u_history_Adam_Pretrain = np.append(loss_f_u_history_Adam_Pretrain, loss_f_u.numpy())
            loss_lambda_u_history_Adam_Pretrain = np.append(loss_lambda_u_history_Adam_Pretrain, loss_lambda_u.numpy())

            lambda_uux_new = np.array([model.lambda_uux.numpy()])
            lambda_uux_history_Adam_Pretrain = np.append(lambda_uux_history_Adam_Pretrain, lambda_uux_new, axis=1)

            start_time = time.time()
    np.save(f'Loss-Coe/{second_path}/loss_history_Adam_Pretrain', loss_history_Adam_Pretrain)
    np.save(f'Loss-Coe/{second_path}/loss_u_history_Adam_Pretrain', loss_u_history_Adam_Pretrain)
    np.save(f'Loss-Coe/{second_path}/loss_f_u_history_Adam_Pretrain', loss_f_u_history_Adam_Pretrain)
    np.save(f'Loss-Coe/{second_path}/loss_lambda_u_history_Adam_Pretrain', loss_lambda_u_history_Adam_Pretrain)

    np.save(f'Loss-Coe/{second_path}/lambda_u_history_Adam_Pretrain', lambda_u_history_Adam_Pretrain)
    np.save(f'Loss-Coe/{second_path}/lambda_uux_history_Adam_Pretrain', lambda_uux_history_Adam_Pretrain)

Run training and save the trained model.

In [ ]:
for epoch, lr in zip(epochs, learning_rate):
    train(model_pretrain, int(epoch), lr)
ms.save_checkpoint(model_pretrain.dnn, f'model/{second_path}/model.ckpt')

It: 0, Loss: 8.652e-01, loss_u:  8.652e-01, loss_f:  4.359e-08, loss_lambda:  0.000e+00, Lambda_uux: 0.000, Lambda_uxx: -0.000, Lambda_uxxxx: -0.000, Time: 9.59
It: 10, Loss: 8.434e-01, loss_u:  8.434e-01, loss_f:  7.940e-05, loss_lambda:  1.668e-09, Lambda_uux: 0.003, Lambda_uxx: 0.003, Lambda_uxxxx: -0.002, Time: 2.70
It: 20, Loss: 8.391e-01, loss_u:  8.390e-01, loss_f:  6.748e-05, loss_lambda:  4.801e-09, Lambda_uux: 0.012, Lambda_uxx: 0.011, Lambda_uxxxx: -0.006, Time: 2.71
It: 30, Loss: 8.277e-01, loss_u:  8.274e-01, loss_f:  3.814e-04, loss_lambda:  7.764e-09, Lambda_uux: 0.022, Lambda_uxx: 0.012, Lambda_uxxxx: -0.012, Time: 2.71
It: 40, Loss: 8.105e-01, loss_u:  8.096e-01, loss_f:  9.378e-04, loss_lambda:  1.053e-08, Lambda_uux: 0.034, Lambda_uxx: 0.001, Lambda_uxxxx: -0.025, Time: 2.70
It: 50, Loss: 7.876e-01, loss_u:  7.863e-01, loss_f:  1.300e-03, loss_lambda:  1.481e-08, Lambda_uux: 0.047, Lambda_uxx: -0.014, Lambda_uxxxx: -0.024, Time: 2.70
It: 60, Loss: 7.637e-01, loss_u: 

Save the learnable parameters of the last training for equation discovery.

In [ ]:
lambda_uux_value = model_pretrain.lambda_uux.numpy()
lambda_u_value = model_pretrain.lambda_u.numpy()
np.save(f'Loss-Coe/{second_path}/lambda_uux_value', lambda_uux_value)
np.save(f'Loss-Coe/{second_path}/lambda_u_value', lambda_u_value)

Directly performing equation discovery after training may exceed the memory of GPU or NPU, so it is necessary to determine whether to reload the model for equation discovery based on the GPU or NPU memory size.

In [ ]:
if load_params:
    lambda_u_value = np.load(f'Loss-Coe/{second_path}/lambda_u_value.npy')
    lambda_uux_value = np.load(f'Loss-Coe/{second_path}/lambda_uux_value.npy')
    model_ICCO = InvarianceConstrainedNN_STRdige(X_u_train, u_train, X_f_train, network_size, BatchNo, lambda_u_value, lambda_uux_value, load_params, second_path, msfloat_type)
else:
    model_ICCO = InvarianceConstrainedNN_STRdige(X_u_train, u_train, X_f_train, network_size, BatchNo, lambda_u_value, lambda_uux_value, load_params, second_path, msfloat_type)

In [ ]:
lambda_u_STRidge = model_ICCO.call_trainstridge(lam, d_tol)

In [ ]:
# GPU results
print_pde(lambda_uux_value, lambda_u_STRidge, description_ks, ut='u_t')

u_t = (0.973947)uu_x
   (-0.967219)u_xx
    + (-0.967183)u_xxxx
   
